In [ ]:
pip install transformers

형태소 단위: https://github.com/ukairia777/tensorflow-bert-ner

In [ ]:
pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=a73ca0e5de0ea0a06a525c19d7ea47701148d5c1d7d70a38cc359ba83387b503
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import pandas as pd
import urllib.request
import numpy as np
import os
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, BertModel
from seqeval.metrics import f1_score, classification_report

# 1. 데이터 로드

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_train_data.csv", filename="ner_train_data.csv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_test_data.csv", filename="ner_test_data.csv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_label.txt", filename="ner_label.txt")

('ner_label.txt', <http.client.HTTPMessage at 0x78115539d930>)

In [ ]:
train_ner_df = pd.read_csv("ner_train_data.csv")

In [ ]:
train_ner_df.head()

,Sentence,Tag
0,"정은 씨를 힘들게 한 가스나그, 가만둘 수 없겠죠 .",PER-B O O O O O O O O
1,▶ 쿠마리 한동수가 말하는 '가넷 & 에르덴',O PER-B PER-I O PER-B O PER-B
2,슈나이더의 프레젠테이션은 말 청중을 위한 특별한 쇼다 .,PER-B O O CVL-B O O O O
3,지구 최대 연료탱크 수검 회사 구글이 연내 22명 안팎의 인력을 갖춘 연구개발(R&...,O O TRM-B O O ORG-B DAT-B NUM-B O O O ORG-B LO...
4,5. <10:00:TI_HOUR> 도이치증권대 <0:1:QT_SPORTS> 연예오락...,NUM-B O ORG-B O ORG-B


In [ ]:
test_ner_df = pd.read_csv("ner_test_data.csv")

In [ ]:
test_ner_df.head()

,Sentence,Tag
0,"라티은-원윤정, 휘닉스파크클래식 프로골퍼",PER-B EVT-B CVL-B
1,5원으로 맺어진 애인까지 돈이라는 민감한 원자재를 통해 현대인의 물질만능주의를 꼬집...,NUM-B O O O O O O O O O O O FLD-B O
2,-날로 삼키면 맛이 어떤지 일차 드셔보시겠어요 .,O O O O NUM-B O O
3,"-네, 지었습니다 .",O O O
4,◇신규 투자촉진에 방점=이번 접속료 조정결과에서 눈에 띄는 지점은 WCDMA/HSD...,O O O O O O O O TRM-B O TRM-B TRM-I ORG-B O TR...


In [ ]:
print("학습 데이터 샘플 개수 :", len(train_ner_df))
print("테스트 데이터 샘플 개수 :", len(test_ner_df))

학습 데이터 샘플 개수 : 81000
테스트 데이터 샘플 개수 : 9000


In [ ]:
train_data_sentence = [sent.split() for sent in train_ner_df['Sentence'].values]
test_data_sentence = [sent.split() for sent in test_ner_df['Sentence'].values]
train_data_label = [tag.split() for tag in train_ner_df['Tag'].values]
test_data_label = [tag.split() for tag in test_ner_df['Tag'].values]

In [ ]:
labels = [label.strip() for label in open('ner_label.txt', 'r', encoding='utf-8')]
print('개체명 태깅 정보 :', labels)

개체명 태깅 정보 : ['O', 'PER-B', 'PER-I', 'FLD-B', 'FLD-I', 'AFW-B', 'AFW-I', 'ORG-B', 'ORG-I', 'LOC-B', 'LOC-I', 'CVL-B', 'CVL-I', 'DAT-B', 'DAT-I', 'TIM-B', 'TIM-I', 'NUM-B', 'NUM-I', 'EVT-B', 'EVT-I', 'ANM-B', 'ANM-I', 'PLT-B', 'PLT-I', 'MAT-B', 'MAT-I', 'TRM-B', 'TRM-I']


In [ ]:
tag_to_index = {tag: index for index, tag in enumerate(labels)}
index_to_tag = {index: tag for index, tag in enumerate(labels)}

In [ ]:
print(tag_to_index)
print(index_to_tag)

{'O': 0, 'PER-B': 1, 'PER-I': 2, 'FLD-B': 3, 'FLD-I': 4, 'AFW-B': 5, 'AFW-I': 6, 'ORG-B': 7, 'ORG-I': 8, 'LOC-B': 9, 'LOC-I': 10, 'CVL-B': 11, 'CVL-I': 12, 'DAT-B': 13, 'DAT-I': 14, 'TIM-B': 15, 'TIM-I': 16, 'NUM-B': 17, 'NUM-I': 18, 'EVT-B': 19, 'EVT-I': 20, 'ANM-B': 21, 'ANM-I': 22, 'PLT-B': 23, 'PLT-I': 24, 'MAT-B': 25, 'MAT-I': 26, 'TRM-B': 27, 'TRM-I': 28}
{0: 'O', 1: 'PER-B', 2: 'PER-I', 3: 'FLD-B', 4: 'FLD-I', 5: 'AFW-B', 6: 'AFW-I', 7: 'ORG-B', 8: 'ORG-I', 9: 'LOC-B', 10: 'LOC-I', 11: 'CVL-B', 12: 'CVL-I', 13: 'DAT-B', 14: 'DAT-I', 15: 'TIM-B', 16: 'TIM-I', 17: 'NUM-B', 18: 'NUM-I', 19: 'EVT-B', 20: 'EVT-I', 21: 'ANM-B', 22: 'ANM-I', 23: 'PLT-B', 24: 'PLT-I', 25: 'MAT-B', 26: 'MAT-I', 27: 'TRM-B', 28: 'TRM-I'}


In [ ]:
tag_size = len(tag_to_index)
print('개체명 태깅 정보의 개수 :',tag_size)

개체명 태깅 정보의 개수 : 29


# 2. 전처리 예시

In [ ]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [ ]:
sent = train_data_sentence[1]
label = train_data_label[1]

print('문장 :', sent)
print('레이블 :',label)
print('레이블의 정수 인코딩 :',[tag_to_index[idx] for idx in label])
print('문장의 길이 :', len(sent))
print('레이블의 길이 :', len(label))

문장 : ['▶', '쿠마리', '한동수가', '말하는', "'가넷", '&', "에르덴'"]
레이블 : ['O', 'PER-B', 'PER-I', 'O', 'PER-B', 'O', 'PER-B']
레이블의 정수 인코딩 : [0, 1, 2, 0, 1, 0, 1]
문장의 길이 : 7
레이블의 길이 : 7


In [ ]:
tokens = []

for one_word in sent:
  # 각 단어에 대해서 서브워드로 분리.
  # ex) one_word = '쿠마리' ===> subword_tokens = ['쿠', '##마리']
  # ex) one_word = '한동수가' ===> subword_tokens = ['한동', '##수', '##가']
  subword_tokens = tokenizer.tokenize(one_word)
  tokens.extend(subword_tokens)

print('BERT 토크나이저 적용 후 문장 :',tokens)
print('레이블 :', label)
print('레이블의 정수 인코딩 :',[tag_to_index[idx] for idx in label])
print('문장의 길이 :', len(tokens))
print('레이블의 길이 :', len(label))

BERT 토크나이저 적용 후 문장 : ['▶', '쿠', '##마리', '한동', '##수', '##가', '말', '##하', '##는', "'", '가', '##넷', '&', '에르', '##덴', "'"]
레이블 : ['O', 'PER-B', 'PER-I', 'O', 'PER-B', 'O', 'PER-B']
레이블의 정수 인코딩 : [0, 1, 2, 0, 1, 0, 1]
문장의 길이 : 16
레이블의 길이 : 7


In [ ]:
tokens = []
labels_ids = []

for one_word, label_token in zip(train_data_sentence[1], train_data_label[1]):
  subword_tokens = tokenizer.tokenize(one_word)
  tokens.extend(subword_tokens)
  labels_ids.extend([tag_to_index[label_token]]+ [-100] * (len(subword_tokens) - 1))

print('토큰화 후 문장 :',tokens)
print('레이블 :', ['[PAD]' if idx == -100 else index_to_tag[idx] for idx in labels_ids])
print('레이블의 정수 인코딩 :', labels_ids)
print('문장의 길이 :', len(tokens))
print('레이블의 길이 :', len(labels_ids))

토큰화 후 문장 : ['▶', '쿠', '##마리', '한동', '##수', '##가', '말', '##하', '##는', "'", '가', '##넷', '&', '에르', '##덴', "'"]
레이블 : ['O', 'PER-B', '[PAD]', 'PER-I', '[PAD]', '[PAD]', 'O', '[PAD]', '[PAD]', 'PER-B', '[PAD]', '[PAD]', 'O', 'PER-B', '[PAD]', '[PAD]']
레이블의 정수 인코딩 : [0, 1, -100, 2, -100, -100, 0, -100, -100, 1, -100, -100, 0, 1, -100, -100]
문장의 길이 : 16
레이블의 길이 : 16


# 3. 전처리

In [ ]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        tokens = []
        labels_ids = []
        for one_word, label_token in zip(example, label):
            # 하나의 단어에 대해서 서브워드로 토큰화
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)

            # 서브워드 중 첫번째 서브워드만 개체명 레이블을 부여하고 그 외에는 -100으로 채운다.
            labels_ids.extend([tag_to_index[label_token]]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        # [CLS]와 [SEP]를 후에 추가할 것을 고려하여 최대 길이를 초과하는 샘플의 경우 max_seq_len - 2의 길이로 변환.
        # ex) max_seq_len = 64라면 길이가 62보다 긴 샘플은 뒷 부분을 자르고 길이 62로 변환.
        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]

        # [SEP]를 추가하는 코드
        # 1. 토큰화 결과의 맨 뒷 부분에 [SEP] 토큰 추가
        # 2. 레이블에도 맨 뒷 부분에 -100 추가.
        tokens += [sep_token]
        labels_ids += [pad_token_id_for_label]

        # [CLS]를 추가하는 코드
        # 1. 토큰화 결과의 앞 부분에 [CLS] 토큰 추가
        # 2. 레이블의 맨 앞 부분에도 -100 추가.
        tokens = [cls_token] + tokens
        labels_ids = [pad_token_id_for_label] + labels_ids

        # 정수 인코딩
        input_id = tokenizer.convert_tokens_to_ids(tokens)

        # 어텐션 마스크 생성
        attention_mask = [1] * len(input_id)

        # 정수 인코딩에 추가할 패딩 길이 연산
        padding_count = max_seq_len - len(input_id)

        # 정수 인코딩, 어텐션 마스크에 패딩 추가
        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        # 세그먼트 인코딩.
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        # 레이블 패딩. (단, 이 경우는 패딩 토큰의 ID가 -100)
        label = labels_ids + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label) == max_seq_len, "Error with labels length {} vs {}".format(len(label), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
X_train, y_train = convert_examples_to_features(train_data_sentence, train_data_label, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 81000/81000 [00:36<00:00, 2247.04it/s]


In [ ]:
len(X_train)

3

In [ ]:
len(y_train)

81000

In [ ]:
print('기존 원문 :', train_data_sentence[0])
print('기존 레이블 :', train_data_label[0])
print('-' * 50)
print('토큰화 후 원문 :', [tokenizer.decode([word]) for word in X_train[0][0]])
print('토큰화 후 레이블 :', ['[PAD]' if idx == -100 else index_to_tag[idx] for idx in y_train[0]])
print('-' * 50)
print('정수 인코딩 결과 :', X_train[0][0])
print('정수 인코딩 레이블 :', y_train[0])

기존 원문 : ['정은', '씨를', '힘들게', '한', '가스나그,', '가만둘', '수', '없겠죠', '.']
기존 레이블 : ['PER-B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
--------------------------------------------------
토큰화 후 원문 : ['[CLS]', '정은', '씨', '##를', '힘들', '##게', '한', '가스', '##나', '##그', ',', '가만', '##둘', '수', '없', '##겠', '##죠', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]'

In [ ]:
print('세그먼트 인코딩 :', X_train[2][0])
print('어텐션 마스크 :', X_train[1][0])

세그먼트 인코딩 : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
X_test, y_test = convert_examples_to_features(test_data_sentence, test_data_label, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 9000/9000 [00:04<00:00, 2170.70it/s]


## 4. nn.CrossEntropy의 -100 무시

In [ ]:
import torch
import torch.nn as nn

# 모델의 예측값
outputs_with_ignore = torch.tensor([[1.0, 2.0, 3.0], # 값이 가장 큰 위치의 인덱스가 정답
                                    [2.0, 1.0, 3.0],
                                    [3.0, 2.0, 1.0],
                                    [1.0, 3.0, 2.0]])
                                    # [2, 2, 0, 1]

# 레이블 (-100인 레이블에 대해서는 오차를 계산하지 않습니다.)
targets_with_ignore = torch.tensor([2, -100, 0, -100])

# -100을 무시하는 설정으로 손실 계산
loss_fn_with_ignore = nn.CrossEntropyLoss(ignore_index=-100)
loss_with_ignore = loss_fn_with_ignore(outputs_with_ignore, targets_with_ignore)

# 오차
print(f'Loss with ignore_index=-100: {loss_with_ignore.item()}')

Loss with ignore_index=-100: 0.40760594606399536


In [ ]:
# 위의 데이터에서 -100이 있는 위치의 값을 실제로 제거한 데이터
# 모델의 예측값
outputs = torch.tensor([[1.0, 2.0, 3.0],
                        [3.0, 2.0, 1.0]])
# 레이블
targets = torch.tensor([2, 0])

loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(outputs, targets)

# 오차
print(f'calculated loss: {loss.item()}')

calculated loss: 0.40760594606399536


# 5. 모델링과 학습

In [ ]:
import torch
from transformers import BertForTokenClassification
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
from seqeval.metrics import f1_score, classification_report

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = BertForTokenClassification.from_pretrained("klue/bert-base", num_labels=tag_size)
optimizer = Adam(model.parameters(), lr=5e-5)
model.to(device)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the mo

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [ ]:
def sequences_to_tags(label_ids, pred_ids, index_to_tag):
    # label_ids: 실제 레이블의 인덱스 시퀀스 리스트 (2D 리스트)
    # pred_ids: 예측된 레이블의 인덱스 시퀀스 리스트 (2D 리스트)
    # index_to_tag: 인덱스를 태그로 변환하는 딕셔너리

    label_list = []
    pred_list = []

    # 각 시퀀스에 대해 반복
    for i in range(0, len(label_ids)):
        label_tag = []  # 현재 시퀀스의 실제 레이블 태그들을 저장할 리스트
        pred_tag = []   # 현재 시퀀스의 예측된 레이블 태그들을 저장할 리스트

        # 각 시퀀스의 레이블 및 예측값 쌍에 대해 반복
        for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
            if label_index != -100:  # 유효하지 않은 레이블 (예: 패딩 등) 제외
                label_tag.append(index_to_tag[label_index])  # 실제 레이블 태그 추가
                pred_tag.append(index_to_tag[pred_index])    # 예측된 레이블 태그 추가

        label_list.append(label_tag)  # 현재 시퀀스의 실제 레이블 태그 리스트를 전체 리스트에 추가
        pred_list.append(pred_tag)    # 현재 시퀀스의 예측된 레이블 태그 리스트를 전체 리스트에 추가

    return label_list, pred_list  # 실제 레이블과 예측된 레이블의 태그 리스트 반환

In [ ]:
def evaluate(model, test_loader, index_to_tag):
    # 모델을 평가 모드로 전환 (드롭아웃 등 비활성화)
    model.eval()
    total_labels, total_preds = [], []  # 전체 레이블과 예측값을 저장할 리스트 초기화

    # 그레디언트 계산을 비활성화하여 메모리 사용량 및 연산 속도를 최적화
    with torch.no_grad():
        # 테스트 데이터셋의 각 배치에 대해 반복
        for input_ids, attention_masks, token_type_ids, labels in test_loader:
            # 데이터를 GPU 또는 지정된 디바이스로 이동
            input_ids = input_ids.to(device)
            attention_masks = attention_masks.to(device)
            token_type_ids = token_type_ids.to(device)
            labels = labels.to(device)

            # 모델에 입력 데이터를 주고 예측값(logits) 출력
            outputs = model(input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids)

            # 예측값(logits)을 CPU로 이동시키고 넘파이 배열로 변환
            logits = outputs.logits.detach().cpu().numpy()

            # 레이블을 CPU로 이동시키고 넘파이 배열로 변환
            labels = labels.cpu().numpy()

            # 예측값에서 가장 높은 확률의 인덱스를 선택하여 예측된 레이블 생성
            y_predicted = np.argmax(logits, axis=2)

            # 실제 레이블과 예측된 레이블을 태그로 변환
            label_list, pred_list = sequences_to_tags(labels, y_predicted, index_to_tag)

            # 전체 레이블 리스트와 예측 리스트에 현재 배치의 결과를 추가
            total_labels.extend(label_list)
            total_preds.extend(pred_list)

    # 전체 레이블과 예측값에 대한 F1 점수를 계산
    score = f1_score(total_labels, total_preds, suffix=True)

    # F1 점수를 출력
    print(' - f1: {:04.2f}'.format(score * 100))

    # 전체 레이블과 예측값에 대한 분류 리포트를 출력
    print(classification_report(total_labels, total_preds, suffix=True))

In [ ]:
batch_size = 32

In [ ]:
# 이 부분은 convert_examples_to_features에서 반환하는 형식에 맞게 텐서를 분할합니다.
# X_train과 y_train은 학습 데이터, X_test와 y_test는 테스트 데이터로 사용됩니다.

# 학습 데이터에서 각 입력 값(input_ids, attention_masks, token_type_ids)과 레이블(labels)을 추출
train_input_ids, train_attention_masks, train_token_type_ids = X_train
train_labels = y_train

# 테스트 데이터에서 각 입력 값(input_ids, attention_masks, token_type_ids)과 레이블(labels)을 추출
test_input_ids, test_attention_masks, test_token_type_ids = X_test
test_labels = y_test

# 학습 데이터의 각 부분을 파이토치 텐서로 변환 (정수형)
train_input_ids = torch.tensor(train_input_ids, dtype=torch.long)
train_attention_masks = torch.tensor(train_attention_masks, dtype=torch.long)
train_token_type_ids = torch.tensor(train_token_type_ids, dtype=torch.long)
train_labels = torch.tensor(train_labels, dtype=torch.long)

# 테스트 데이터의 각 부분을 파이토치 텐서로 변환 (정수형)
test_input_ids = torch.tensor(test_input_ids, dtype=torch.long)
test_attention_masks = torch.tensor(test_attention_masks, dtype=torch.long)
test_token_type_ids = torch.tensor(test_token_type_ids, dtype=torch.long)
test_labels = torch.tensor(test_labels, dtype=torch.long)

# 학습 데이터와 테스트 데이터 텐서들을 하나의 TensorDataset으로 묶음
train_data = TensorDataset(train_input_ids, train_attention_masks, train_token_type_ids, train_labels)
test_data = TensorDataset(test_input_ids, test_attention_masks, test_token_type_ids, test_labels)

# 학습 데이터와 테스트 데이터에 대해서 데이터 로더를 생성, batch_size로 데이터를 묶어 모델에 전달할 준비를 함
train_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
import tqdm

steps = len(train_input_ids) // batch_size + 1
print(steps)

2532


In [ ]:
EPOCHS = 3  # 학습을 반복할 횟수(에포크 수)를 설정

# 설정한 에포크 수만큼 반복
for epoch in range(EPOCHS):
    model.train()  # 모델을 학습 모드로 전환 (드롭아웃 등 활성화)

    # 학습 데이터 로더에서 배치를 하나씩 가져와서 반복
    for input_ids, attention_masks, token_type_ids, labels in tqdm.tqdm(train_loader, total=steps):
        # 각 입력 데이터를 GPU 또는 지정된 디바이스로 이동
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = labels.to(device)

        # 옵티마이저의 그레디언트를 초기화
        optimizer.zero_grad()

        # 모델에 데이터를 입력하여 예측값(outputs)을 계산하고 손실(loss)도 계산
        outputs = model(input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids, labels=labels)
        loss = outputs.loss  # 손실 값 추출

        # 역전파를 통해 그레디언트를 계산
        loss.backward()

        # 옵티마이저를 통해 모델 파라미터를 업데이트
        optimizer.step()

    # 한 에포크가 끝난 후, 모델을 평가하여 성능을 측정
    evaluate(model, test_loader, index_to_tag)

100%|██████████| 2532/2532 [27:34<00:00,  1.53it/s]


 - f1: 84.80
              precision    recall  f1-score   support

         AFW       0.69      0.57      0.62       394
         ANM       0.81      0.64      0.71       701
         CVL       0.81      0.84      0.82      5758
         DAT       0.89      0.92      0.90      2521
         EVT       0.74      0.74      0.74      1094
         FLD       0.78      0.44      0.56       228
         LOC       0.87      0.83      0.85      2126
         MAT       0.12      0.08      0.10        12
         NUM       0.90      0.92      0.91      5590
         ORG       0.84      0.88      0.86      4086
         PER       0.85      0.91      0.88      4426
         PLT       1.00      0.06      0.11        34
         TIM       0.80      0.91      0.85       314
         TRM       0.77      0.71      0.74      1964

   micro avg       0.84      0.85      0.85     29248
   macro avg       0.78      0.68      0.69     29248
weighted avg       0.84      0.85      0.85     29248



100%|██████████| 2532/2532 [27:33<00:00,  1.53it/s]


 - f1: 85.59
              precision    recall  f1-score   support

         AFW       0.66      0.58      0.62       394
         ANM       0.79      0.73      0.76       701
         CVL       0.83      0.85      0.84      5758
         DAT       0.90      0.93      0.91      2521
         EVT       0.77      0.75      0.76      1094
         FLD       0.75      0.57      0.65       228
         LOC       0.87      0.85      0.86      2126
         MAT       0.23      0.25      0.24        12
         NUM       0.91      0.93      0.92      5590
         ORG       0.84      0.89      0.86      4086
         PER       0.87      0.91      0.89      4426
         PLT       0.50      0.18      0.26        34
         TIM       0.84      0.89      0.86       314
         TRM       0.74      0.74      0.74      1964

   micro avg       0.85      0.86      0.86     29248
   macro avg       0.75      0.72      0.73     29248
weighted avg       0.85      0.86      0.85     29248



100%|██████████| 2532/2532 [27:33<00:00,  1.53it/s]


 - f1: 85.82
              precision    recall  f1-score   support

         AFW       0.66      0.61      0.63       394
         ANM       0.76      0.76      0.76       701
         CVL       0.84      0.84      0.84      5758
         DAT       0.91      0.92      0.92      2521
         EVT       0.77      0.77      0.77      1094
         FLD       0.67      0.61      0.64       228
         LOC       0.87      0.84      0.86      2126
         MAT       0.15      0.17      0.16        12
         NUM       0.91      0.93      0.92      5590
         ORG       0.82      0.90      0.86      4086
         PER       0.87      0.91      0.89      4426
         PLT       0.57      0.12      0.20        34
         TIM       0.81      0.92      0.86       314
         TRM       0.75      0.74      0.75      1964

   micro avg       0.85      0.87      0.86     29248
   macro avg       0.74      0.72      0.72     29248
weighted avg       0.85      0.87      0.86     29248



# 6. 예측

In [ ]:
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, label_masks = [], [], [], []

    for example in tqdm.tqdm(examples):
        tokens = []
        label_mask = []
        for one_word in example:
            # 하나의 단어에 대해서 서브워드로 토큰화
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)\
            # 서브워드 중 첫번째 서브워드를 제외하고 그 뒤의 서브워드들은 -100으로 채운다.
            label_mask.extend([0]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        # [CLS]와 [SEP]를 후에 추가할 것을 고려하여 최대 길이를 초과하는 샘플의 경우 max_seq_len - 2의 길이로 변환.
        # ex) max_seq_len = 64라면 길이가 62보다 긴 샘플은 뒷 부분을 자르고 길이 62로 변환.
        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            label_mask = label_mask[:(max_seq_len - special_tokens_count)]

        # [SEP]를 추가하는 코드
        # 1. 토큰화 결과의 맨 뒷 부분에 [SEP] 토큰 추가
        # 2. 레이블에도 맨 뒷 부분에 -100 추가.
        tokens += [sep_token]
        label_mask += [pad_token_id_for_label]

        # [CLS]를 추가하는 코드
        # 1. 토큰화 결과의 앞 부분에 [CLS] 토큰 추가
        # 2. 레이블의 맨 앞 부분에도 -100 추가.
        tokens = [cls_token] + tokens
        label_mask = [pad_token_id_for_label] + label_mask
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_id)

        # 정수 인코딩에 추가할 패딩 길이 연산
        padding_count = max_seq_len - len(input_id)

         # 정수 인코딩, 어텐션 마스크에 패딩 추가
        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)

        # 세그먼트 인코딩.
        token_type_id = [pad_token_id_for_segment] * max_seq_len

        # 레이블 패딩. (단, 이 경우는 패딩 토큰의 ID가 -100)
        label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label_mask) == max_seq_len, "Error with labels length {} vs {}".format(len(label_mask), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        label_masks.append(label_mask)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    label_masks = np.asarray(label_masks, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), label_masks

In [ ]:
test_data_sentence[:5]

[['라티은-원윤정,', '휘닉스파크클래식', '프로골퍼'],
 ['5원으로',
  '맺어진',
  '애인까지',
  '돈이라는',
  '민감한',
  '원자재를',
  '통해',
  '현대인의',
  '물질만능주의를',
  '꼬집고',
  '있는',
  '이',
  '무비는',
  '.'],
 ['-날로', '삼키면', '맛이', '어떤지', '일차', '드셔보시겠어요', '.'],
 ['-네,', '지었습니다', '.'],
 ['◇신규',
  '투자촉진에',
  '방점=이번',
  '접속료',
  '조정결과에서',
  '눈에',
  '띄는',
  '지점은',
  'WCDMA/HSDPA',
  '등',
  '3세대(G)',
  '초고속인터넷',
  '뉴스핌',
  '의',
  '아치형(BcN)',
  '출자분을',
  '적극',
  '반영했다는',
  '점이다',
  '.']]

In [ ]:
X_pred, label_masks = convert_examples_to_features_for_prediction(test_data_sentence[:5], max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 5/5 [00:00<00:00, 798.15it/s]


In [ ]:
print('기존 원문 :', test_data_sentence[0])
print('-' * 50)
print('토큰화 후 원문 :', [tokenizer.decode([word]) for word in X_pred[0][0]])
print('레이블 마스크 :', ['[PAD]' if idx == -100 else '[FIRST]' for idx in label_masks[0]])

기존 원문 : ['라티은-원윤정,', '휘닉스파크클래식', '프로골퍼']
--------------------------------------------------
토큰화 후 원문 : ['[CLS]', '라', '##티', '##은', '-', '원', '##윤', '##정', ',', '휘', '##닉스', '##파크', '##클', '##래', '##식', '프로', '##골', '##퍼', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[P

In [ ]:
def ner_prediction(examples, max_seq_len, tokenizer, model, device):
    examples = [sent.split() for sent in examples]
    X_pred, label_masks = convert_examples_to_features_for_prediction(examples, max_seq_len=128, tokenizer=tokenizer)

    # Convert input_ids, attention_masks, and token_type_ids to tensors
    input_ids, attention_masks, token_type_ids = X_pred
    input_ids = torch.tensor(input_ids, dtype=torch.long).to(device)
    attention_masks = torch.tensor(attention_masks, dtype=torch.long).to(device)
    token_type_ids = torch.tensor(token_type_ids, dtype=torch.long).to(device)

    label_masks = torch.tensor(label_masks, dtype=torch.long).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids)
        logits = outputs.logits.detach().cpu().numpy()
        y_predicted = np.argmax(logits, axis=2)

    pred_list = []
    result_list = []

    # ex) 모델의 예측값 디코딩 과정
    # 예측값(y_predicted)에서 레이블 마스크(label_masks)의 값이 -100인 동일 위치의 값을 삭제
    # label_masks : [-100 0 -100 0 -100]
    # y_predicted : [  0  1   0  2   0 ] ==> [1 2] ==> 최종 예측(pred_tag) : [PER-B PER-I]
    for i in range(0, len(label_masks)):
        pred_tag = []
        for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
            if label_index != -100:
                pred_tag.append(index_to_tag[pred_index])

        pred_list.append(pred_tag)

    for example, pred in zip(examples, pred_list):
        one_sample_result = []
        for one_word, label_token in zip(example, pred):
            one_sample_result.append((one_word, label_token))
        result_list.append(one_sample_result)

    return result_list

In [ ]:
sent1 = '오리온스는 리그 최정상급 포인트가드 김동훈을 앞세우는 빠른 공수전환이 돋보이는 팀이다'
sent2 = '하이신사에 속한 섬들도 위로 솟아 있는데 타인은 살고 있어요'

In [ ]:
test_samples = [sent1, sent2]

In [ ]:
result_list = ner_prediction(test_samples, max_seq_len=128, tokenizer=tokenizer, model=model, device=device)

100%|██████████| 2/2 [00:00<00:00, 306.44it/s]


In [ ]:
result_list

[[('오리온스는', 'ORG-B'),
  ('리그', 'O'),
  ('최정상급', 'O'),
  ('포인트가드', 'CVL-B'),
  ('김동훈을', 'PER-B'),
  ('앞세우는', 'O'),
  ('빠른', 'O'),
  ('공수전환이', 'O'),
  ('돋보이는', 'O'),
  ('팀이다', 'O')],
 [('하이신사에', 'LOC-B'),
  ('속한', 'O'),
  ('섬들도', 'O'),
  ('위로', 'O'),
  ('솟아', 'O'),
  ('있는데', 'O'),
  ('타인은', 'O'),
  ('살고', 'O'),
  ('있어요', 'O')]]

In [ ]:
# my_bert_model이라는 폴더가 생기고 그 안에 모델 파일들이 저장됨.
model_save_path = "./my_bert_model"
model.save_pretrained(model_save_path)

model = BertForTokenClassification.from_pretrained(model_save_path)